In [ ]:
# default_exp machine_learning

# Machine Learning

> Apply simple machine learning algorithms to a measures table

In [ ]:
#hide
from nbdev.showdoc import *

The gamba library contains several wrapper methods for machine learning methods...